In [14]:
import warnings
warnings.filterwarnings("ignore")
from utils import *
import pickle
OPBG = [ 'age at diagnosis (months)',            
            'ntbc dosis mg/kg/day',
            'ntbc levels (dbs)',
            'sca (urine)',
            'methionine dbs',
            'tyrosine dbs',
            'phenylalanine dbs',
            'methionine (plasma)',
            'tyrosine (plasma)',
            'phenylalanine (plasma)',
            'inr',
            'pt (sec)',
            'bili total',
            'gpt',
            'got',
            'ggt',
            'alkaline phosphatase',
            'alfa-fetoprotein']
    
df1 = pd.read_csv('data/tirosinemia.csv').loc[:,OPBG]
df2 = pd.read_csv('data/tirosinemia_italia.csv').loc[:,OPBG]
compare_dataframes(df1, df2)
def extract_transform(path, keep_cols, cols_rows_with_missing_values, new_binary_target):
    _ , spark_df = SparkDataProcessor().process_data(
        data_csv_path= path,
        )

    return prepare_spark_df(
        spark_df=spark_df,
        keep_cols=keep_cols,
        cols_rows_with_missing_values=cols_rows_with_missing_values,
        new_binary_target = new_binary_target,
    )

binary_target = 'Alpha-Fet'
df_train = extract_transform("data/tirosinemia.csv", OPBG,  ['alfa-fetoprotein', 'sca (urine)', 'ntbc levels (dbs)'],binary_target)
df_test  = extract_transform("data/tirosinemia_italia.csv", OPBG,  ['alfa-fetoprotein', 'sca (urine)', 'ntbc levels (dbs)'],binary_target)
ray_df_train = ray.put(df_train)
ray_df_test  = ray.put(df_test)

Both CSV files have the same number of columns: 18
Column names are the same.
Column order is the same.
All common columns have the same data types.


In [15]:
features : list[str] = df_train.columns.tolist()
features.remove(binary_target)


In [18]:
%%time

studies = make_multiple_studies(
    ray_df_train, 
    features = features,
    targets = [binary_target],
    n_trials=50,
    Independent_testset = True,
    Independent_testset_df = ray_df_test
)

studies = ray.get(studies)

(make_a_study pid=1559897) [I 2023-06-01 22:21:24,096] A new study created in memory with name: age at diagnosis (months)_predicts_Alpha-Fet
(make_a_study pid=1559888) [I 2023-06-01 22:21:24,096] A new study created in memory with name: age at diagnosis (months)_predicts_Alpha-Fet
(make_a_study pid=1559897) [I 2023-06-01 22:20:35,502] Trial 9 finished with values: [0.5625, 0.4319020211696625] and parameters: {'max_depth': 5, 'eta': 0.297038894706459, 'subsample': 0.4944462533167917, 'lambda': 0.6598217742234079, 'alpha': 0.5066863576385843, 'scale_pos_weight': 2.7659090073301136, 'kfold_splits': 4}.  [repeated 17x across cluster]
(make_a_study pid=1559888) [I 2023-06-01 22:21:24,096] A new study created in memory with name: ntbc dosis mg/kg/day_predicts_Alpha-Fet
(make_a_study pid=1559897) [I 2023-06-01 22:21:29,183] Trial 2 finished with values: [0.9125000000000001, 0.0] and parameters: {'max_depth': 2, 'eta': 0.359920756103038, 'subsample': 0.1248800603956834, 'lambda': 0.66831733734

CPU times: user 7.16 s, sys: 7.39 s, total: 14.6 s
Wall time: 3min 45s


In [20]:
with open('studies_independent_testset.pickle', 'wb') as handle:
    pickle.dump(studies, handle, protocol=pickle.HIGHEST_PROTOCOL)    